In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# DATA = '/home/tsdhrm/Desktop/T/dataset/100Bro/negative'
DATA = '/home/tsdhrm/Desktop/moiOneHundred-20220915T100556Z-001/moiOneHundred'

In [3]:
retina = {}
for fname in os.listdir(DATA):
    f = fname.split('.')[0]
    retina[f] = cv2.imread(os.path.join(os.path.join(DATA, fname)))
    retina[f] = cv2.cvtColor(retina[f], cv2.COLOR_BGR2RGB)

error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


# Functions: Display

In [ ]:
def display_all(imgs, subs=(10,10), use_gray=False):
    fig, axs = plt.subplots(subs[0], subs[1], figsize=(18, 21))
    k = 0
    for i in range(subs[0]):
        for j in range(subs[1]):
            axs[i, j].imshow(list(imgs.values())[k], 'gray' if use_gray else None)
            axs[i, j].set_title(list(imgs.keys())[k])
            axs[i, j].axis('off')
            k+=1
            
def display_all_with(imgs, cc, subs=(10,10), use_gray=True):
    fig, axs = plt.subplots(subs[0], subs[1], figsize=(18, 21))
    k = 0
    for i in range(subs[0]):
        for j in range(subs[1]):
            axs[i, j].imshow(list(imgs.values())[k][:,:,cc], cmap='gray' if use_gray else None)
            axs[i, j].set_title(list(imgs.keys())[k])
            axs[i, j].axis('off')
            k+=1

# Functions: Preprocess

In [ ]:
def resize_all(imgs, prcnt=50):
    resized = {}
    for img in imgs:
        resized[img] = cv2.resize(imgs[img], (int(imgs[img].shape[1]*prcnt/100), int(imgs[img].shape[0]*prcnt/100)), cv2.INTER_AREA)
    return resized

def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Main

In [ ]:
resized = resize_all(retina)

In [ ]:
display_all(resized, (10,10))

# Functions: Get ROI

In [ ]:
def correct_illum(img):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    clahe = cv2.createCLAHE(2.5, (8,8))
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
                                 

def get_roi(imgs):
    im = {}
    for img in imgs:
        ill_corrected= correct_illum(imgs[img])
        ret, im[img] = cv2.threshold(ill_corrected[:,:,0], 125, 255, cv2.THRESH_BINARY)
    display_all(im, (10,10), use_gray=True)
    return 0

In [ ]:
rois = get_roi(resized)